# Advanced Certification in AIML
## A Program by IIIT-H and TalentSprint

In [0]:
#@title Experiment Explanation Video
from IPython.display import HTML

HTML("""<video width="500" height="300" controls>
  <source src="https://cdn.talentsprint.com/aiml/AIML_BATCH_HYD_7/Apr15/HackathonTaskArchitecture.mp4" type="video/mp4">
</video>
""")

# Hackathon 3: Anti Face Spoofing

### Setup Steps

In [0]:
#@title Please enter your registration id to start: (e.g. P181900101) { run: "auto", display-mode: "form" }
Id = "P181902118" #@param {type:"string"}


In [0]:
#@title Please enter your password (normally your phone number) to continue: { run: "auto", display-mode: "form" }
password = "8860303743" #@param {type:"string"}


In [4]:
#@title Run this cell to complete the setup for this Notebook

from IPython import get_ipython
ipython = get_ipython()
  
notebook="M2_Hackathon" #name of the notebook

def setup():
#  ipython.magic("sx pip3 install torch")
    ipython.magic("sx wget https://cdn.talentsprint.com/aiml/FaceRecogHackathon/Datasets/IMFDB_face_recog/IMFDB_final-20190607T024441Z-001.zip")
    ipython.magic("sx wget wget https://cdn.talentsprint.com/aiml/Experiment_related_data/Expression_data.zip")
    ipython.magic("sx wget https://cdn.talentsprint.com/aiml/FaceRecogHackathon/Datasets/ATandT/data-20190607T005435Z-001.zip")
    ipython.magic("sx unzip IMFDB_final-20190607T024441Z-001.zip")
    ipython.magic("sx unzip Expression_data.zip")
    ipython.magic("sx unzip data-20190607T005435Z-001.zip")
    ipython.magic("sx pip install torch==1.0.1 -f https://download.pytorch.org/whl/cu100/stable")
    ipython.magic("sx pip install torchvision==0.2.1")
    ipython.magic("sx pip install opencv-python")
    print ("Setup completed successfully")
    return

def submit_notebook():
    
    ipython.magic("notebook -e "+ notebook + ".ipynb")
    
    import requests, json, base64

    url = "https://dashboard.talentsprint.com/xp/app/save_notebook_attempts"
    if not submission_id:
      data = {"id" : getId(), "notebook" : notebook, "mobile" : getPassword()}
      r = requests.post(url, data = data)
      r = json.loads(r.text)

      if r["status"] == "Success":
          return r["record_id"]
      elif "err" in r:        
        print(r["err"])
        return None        
      else:
        print ("Something is wrong, the notebook will not be submitted for grading")
        return None

    elif getComplexity() and getAdditional() and getConcepts():
      f = open(notebook + ".ipynb", "rb")
      file_hash = base64.b64encode(f.read())

      data = {"complexity" : Complexity, "additional" :Additional, 
              "concepts" : Concepts, "record_id" : submission_id, 
              "id" : Id, "file_hash" : file_hash, "notebook" : notebook}

      r = requests.post(url, data = data)
      print("Your submission is successful. Ref:", submission_id)
      return submission_id
    else: submission_id
    

def getAdditional():
  try:
    if Additional: return Additional      
    else: raise NameError('')
  except NameError:
    print ("Please answer Additional Question")
    return None

def getComplexity():
  try:
    return Complexity
  except NameError:
    print ("Please answer Complexity Question")
    return None
  
def getConcepts():
  try:
    return Concepts
  except NameError:
    print ("Please answer Concepts Question")
    return None

def getId():
  try: 
    return Id if Id else None
  except NameError:
    return None

def getPassword():
  try:
    return password if password else None
  except NameError:
    return None

submission_id = None
### Setup 
if getPassword() and getId():
  submission_id = submit_notebook()
  if submission_id:
    setup()
  
else:
  print ("Please complete Id and Password cells before running setup")



Setup completed successfully


In [0]:
import torch
from torch.autograd import Variable
import numpy as np
import os
import warnings
from time import sleep
import sys
warnings.filterwarnings('ignore')

##Data Collection

* Using the Mobile Application, please collect the Faces and Expression data of your team. 
* These will be stored in the Server to which login is provided to you.
* Download the data into your colab server using the downloadable link provided to you
* This data will be usefull training the networks


##Data used: Indian Movie Face database (IMFDB)
It is a large unconstrained face database of images of 100 Indian actors collected from more than 100 videos. All the images are manually selected and cropped from the video frames resulting in a high degree of variability interms of scale, pose, expression, illumination, age, resolution, occlusion, and makeup

During the setup you have downloaded this data:

* **IMFDB_final**: This folder contains the face images segregrated by the actor in each folder


> * Each class is organised as one folder


* **Expression_data**: In this folder, the same images are segregrated in terms of Expression
> * Expressions available: ANGER, DISGUST, FEAR, HAPPINESS, NEUTRAL, SADNESS, SURPRISE
> * Each class is organised as one folder



In [7]:
%ls

data/                          IMFDB_final-20190607T024441Z-001.zip
data-20190607T005435Z-001.zip  M2_Hackathon.ipynb
Expression_data/               __MACOSX/
Expression_data.zip            sample_data/
IMFDB_final/


###Download your data using the links provided to you

#### NOTE: Replace the string "username" with your team name (such as b7h3gxx) in both the cells below

In [0]:
!wget -nH --recursive --no-parent --reject 'index.*' https://aiml-sandbox.talentsprint.com/expression_detection/username/captured_face_images/ --cut-dirs=3  -P ./captured_face_images


In [0]:
!wget -nH --recursive --no-parent --reject 'index.*' https://aiml-sandbox.talentsprint.com/expression_detection/username/captured_images_with_Expression/ --cut-dirs=3  -P ./captured_images_with_Expression

Once downloaded using 'mv', please add the data into the respective folders IMFDB_final

In [0]:
%ls

##Stage 1 (Face Recognition): (20 marks) 

* From the provided AT&T Faces data or IMFDB data train the Siamese network.
** If you're using the AT&T faces dataset, you are recommended to following the Siamese blog post provided to you last week as the starter code. (Its recommended you quickly try this approach before (if at all) trying the next IMFDB approach) Link [here](https://drive.google.com/open?id=17-hLbZgwgHFMKXHhrDwzWv42fXELMPkn)
** If you're however planning to use IMFDB data, the [ME40_siamese.ipynb](https://drive.google.com/open?id=1bav8yQF28v5rPoM5PbEIE13sPce8SA0K) is recommended as the starter code; Here select any ten celebrity face images (100 classes is too big) and add one of your team member’s data (11th celebrity :p ) using the mobile app.

* Define and train a Siamese network

* Save the state dictionary of the Siamese network (use pytorch only), It will be useful in
integrating to the mobile app

* Define and train a classifier which takes output from the above trained Siamese network
(feature extraction network) as input to classify the above 11 classes

* Save the state dictionary of the classifier (use pytorch only), It will be useful in
integrating to the mobile app

* “face_recognition.py” routine in the server contains the necessary skeleton, integrate your model in
this to predict the face and to get similarity measure.(Note: To define the architecture of your trained model, you'll need to define it in the file "face_recognition_model.py")

* .Upload your files to the given ftp server and test your model on the mobile app

* . **Grading Scheme:**


> * Face Similarity (10M): The face similarity should close similar faces and should be
distant for dissimilar faces using the mobile app’s “Face Similarity” functionality


> * Face Recognition (10M): Recognize the person correctly using the mobile app’s “Face Recognition” functionality

####Save your model

* Save the state dictionary of the classifier (use pytorch only), It will be useful in
integrating model to the mobile app

#### Download your trained model using the code below
* given the path of model file the following code downloads it through the browser

In [0]:
from google.colab import files
files.download('<model_file_path>')

##Stage 2 (Expression Recognition): (20 marks)
* Define and train a CNN for expression recognition with the given IMFDB data segregated
on expression basis.
* Collect your data using mobiledata and fine-tune the CNN for expression data on your team
* “exp_recognition.py” routine contains the necessary skeleton, integrate your model to predict the expression on the face (Note: To define the architecture of your trained model, you'll need to define it in the file "exp_recognition_model.py")
* Upload your files to the given ftp server and test your model on the mobile app
* Grading Scheme:
> * Expression Recognition (10M): If the functionality of giving back an expression
class for the face using the mobile app’s “Expression Recognition” functionality
> * Sequence Expression (10M): Get three consecutive correct Expressions using the
mobile app’s “Sequence Expressions” functionality

In [0]:
##YOUR CODE HERE

####Save your model

* Save the state dictionary of the classifier (use pytorch only), It will be useful in
integrating model to the mobile app

#### Download your trained model using the code below
* given the path of model file the following code downloads it through the browser

In [0]:
from google.colab import files
files.download('<model_file_path>')


***Stage 3 (Anti Face Spoofing): (10 marks)***
The objective of anti-spoofing, is to be able to unlock (say) a screen not just by your image
(which can be easily be spoofed with a photograph of yours) but by a switch in the expression
demanded by the Mobile App (which is much less probable to mimic)
* **Grading scheme**:
> * **Anti Face Spoofing**: (10M Only if both the cases mentioned below are achieved)
>>* **Unlock**: Correct face + Correct Demanded Expression
>>* **Stay Locked**: Correct face + Incorrect Demanded Expression (as you might imagine there are multiple other such possibilities, which you are free to explore)

### Deployment instruction are given the Hackathon documentation

### Please answer the questions below to complete the experiment:

In [0]:
#@title How was the experiment? { run: "auto", form-width: "500px", display-mode: "form" }
Complexity = "" #@param ["Too Simple, I am wasting time", "Good, But Not Challenging for me", "Good and Challenging me", "Was Tough, but I did it", "Too Difficult for me"]


In [0]:
#@title If it was very easy, what more you would have liked to have been added? If it was very difficult, what would you have liked to have been removed? { run: "auto", display-mode: "form" }
Additional = "" #@param {type:"string"}

In [0]:
#@title Can you identify the concepts from the lecture which this experiment covered? { run: "auto", vertical-output: true, display-mode: "form" }
Concepts = "" #@param ["Yes", "No"]

In [0]:
#@title Run this cell to submit your notebook for grading { vertical-output: true }
try:
  if submission_id:
      return_id = submit_notebook()
      if return_id : submission_id =return_id
  else:
      print("Please complete the setup first.")
except NameError:
  print ("Please complete the setup first.")